# Thompson Sampling (not in the Bernoulli case)

Here, I discuss the form of an agent (and Bandit) suitable when there is a lognormal posterior for the expected value of the distribution of the payback when a lever is pulled. This is following chapter 4 of [this book](https://web.stanford.edu/~bvr/pubs/TS_Tutorial.pdf), and uses this specific example to demonstrate how this works. 

(May restructure classes again for better generalisation).

## **The Problem**

Imagine we want to solve an online routing problem. The aim is to get from one side to the other of this graph while incurring the minimum time penalty. The challenge is that we do not know precisely how long traversing any of the edges will take because they are drawn from *as yet unknown* distributions with underlying mean $\theta_{edge}$. 

Were the edge's distributions known, then you would just find the path that minimised $\theta_{t=1, e} + \theta_{t=2, e} + \theta_{t=3, e} + ... $ 

However, you just don't have access to that information. All you can do is try a path and intelligently explore while trying to keep the cost incurred to a minimum. 

In [3]:
from IPython.display import Image

In [ ]:
![title](img/picture.png)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
#general parent class for an agent that is playing a bandit problem.

class StandardAgent:
    
    def __init__(self, bandit):
        #only require the bandit to initialise
        
        #general attributes of all agents
        self.bandit = bandit
        self.pull_record = [0] * self.bandit.k
        self.action_sequence = []
        
        #for keeping track of how the agent learns
        self.num_optimal_pulls = 0
        self.optimal_trajectory = []
        
        #estimated Q is the value of each action, or the expected reward of each action
        self.estimatedQ = [0] * self.bandit.k
        
        self.Q_trajectory = [[] for i in range(self.bandit.k)]
        
        #keep track of the agent's knowledge of the probability distributions
        #cumulative regret
        self.total_regret = 0
        #keeping an eye on how regret changes
        self.regrets_trajectory = []
        
    def update_R(self, lever):
        
        self.total_regret += (self.bandit.best_reward - self.bandit.expected_reward[lever])
        self.regrets_trajectory.append(self.total_regret)
        
    def choose_lever(self):
        #in child classes this involves getting back the lever and returning the reward
        raise NotImplementedError('Not a method in this parent class.')
        
    @property
    def Q_values(self):
        raise NotImplementedError('Not a method in this parent class.')
        
    def run_trial(self, n_steps):
        for step in range(n_steps):
            
            lever = self.choose_lever()
            
            # Update the array keeping track of how many times each lever has been pulled
            self.pull_record[lever] += 1
            
            # update whether lever was best
            if lever == self.bandit.best_arm:
                self.num_optimal_pulls += 1
            self.optimal_trajectory.append(self.num_optimal_pulls / np.sum(self.pull_record))
            
            self.action_sequence.append(lever)
            # update the regret 
            self.update_R(lever)
    

In [ ]:
#child class that can perform Thompson sampling, with lognormal dist prior.

class logNormalThompsonAgent(StandardAgent):
    def __init__(self, bandit, mu = 0, sigma = 100):
        super(logNormalThompsonAgent, self).__init__(bandit)
        
        #sample from the prior distribution over the parameters for each arm (default uninformative prior)
        self.theta_estimate = [np.random.lognormal(mu, sigma) for k in self.bandit.k]
        
    def choose_lever(self):
        
        #using beta prior with initial uniform distribution in first pass
        for i in range(0, self.bandit.k):
            self.theta_estimate[i] = np.random.beta(self.a[i], self.b[i])
        
        #choose the largest resulting theta estimate
        lever = np.argmax(self.theta_estimate)
        reward = self.bandit.pull_lever(lever)
        
        #update the posterior distribution over theta (update parameters of that dist)
        self.a[lever] += reward
        self.b[lever] += 1-reward
        
        self.update_Q()
        
        return lever
    
    def update_Q(self):
        #update the Q values
        for k in range(0, self.bandit.k):
            self.estimatedQ[k] = self.a[k]/(self.a[k] + self.b[k]) #estimate of the Q values for each lever, here they are expectation values
            #print(self.Q_trajectory[k])
            self.Q_trajectory[k].append(self.estimatedQ[k])
    
        
    #record the reward as an update to the 'Q value' for this action:
    @property
    def Q_values(self):
        return self.estimatedQ
        